In [1]:
%load_ext dotenv
%dotenv

In [1]:
import logging
import os
from zeta_py.types import Asset, Side, OrderOptions, OrderType
from zeta_py import utils
from zeta_py.client import Client
import anchorpy
from datetime import datetime, timedelta
from solana.rpc.commitment import Confirmed

logger = logging.getLogger("zeta_py")
logging.basicConfig(level=logging.DEBUG)

endpoint = os.getenv("ENDPOINT", "https://api.mainnet-beta.solana.com")
print(endpoint.split(".com")[0])

wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet
# print(wallet.public_key)

https://api.mainnet-beta.solana


In [2]:
asset = Asset.SOL
client = await Client.load(endpoint=endpoint, commitment=Confirmed, wallet=wallet, assets=[asset])

### Balances, Positions and Open Orders

In [3]:
balance = await client.fetch_balance()
position = await client.fetch_position(asset)
open_orders = await client.fetch_open_orders(asset)

print(f"Balance: {balance}")
print(f"Position: {position}")
print(f"Open orders: {open_orders}")

Balance: 0.489279
Position: Position(size=0.004, cost_of_trades=0.076306)
Open orders: []


### Deposit
Deposit USDC into your Zeta margin account

In [5]:
await client.deposit(0.1)

INFO:zeta_py.client.Client:Depositing 0.1 USDC to margin account


Signature(
    2Sf2dMvGEiM82u6cGo9PYKPfqSMDwgnZaKkAjAaXk8HzRFP3xCk2wJw7pyS4xejKkpu7tYnTFsiCZTtC9W5iUmwN,
)

### Place Order

In [4]:
await client.place_order(asset=Asset.SOL, price=0.1, size=0.001, side=Side.Bid)

INFO:zeta_py.client.Client:Placed 0.001x SOL-PERP Ask @ $0.1
INFO:solanaweb3.rpc.httprpc.HTTPClient:Transaction sent to https://api.mainnet-beta.solana.com. Signature YEUYDVjLguXn1zqtHMbuhnwbQQ3s8F7McA9bPD4rpcqxAQS4KMBbvrKPDicPQFkozfu8nHDFaVqAjZgSWDpwG2R: 


Signature(
    YEUYDVjLguXn1zqtHMbuhnwbQQ3s8F7McA9bPD4rpcqxAQS4KMBbvrKPDicPQFkozfu8nHDFaVqAjZgSWDpwG2R,
)

### Cancel Order

In [7]:
open_orders = await client.fetch_open_orders(Asset.SOL)
print("Open Orders:")
for order in open_orders:
    print(f"{order.side.name} {order.info.size}x ${order.info.price}")

Open Orders:
Bid 0.001x $0.1


In [10]:
oid = open_orders[0].order_id
await client.cancel_order(Asset.SOL, order_id=oid, side=Side.Bid)

INFO:zeta_py.client.Client:Cancelling order 1844692854115028755624221 for SOL


Signature(
    RSyeLPuE4y5UhKWsH88iyEQDQWRexW64Mk2ay3ByPfQanRh4W2mcNkV4syoEnKQpdN1MPD56wCLF8G8zT2fHNRf,
)

### Quoting
Use this code to atomically cancel and replace a set of quotes on Zeta.

*For this example we quote 0.001 SOL-PERP contracts at 20bps spread*

In [12]:
# Quoting parameters
asset = Asset.SOL  # SOL-PERP market
quote_size = 0.001  # min size
quote_bps = 10  # basis points from midpoint
time_in_force = 10  # expire the quote after 10 seconds

# Get the latest mark price and calculate bid/ask prices
price_fixed = client.exchange.pricing.mark_prices[asset.to_index()]
bid_price = utils.convert_fixed_int_to_decimal(price_fixed * (1 - quote_bps / 10000))
ask_price = utils.convert_fixed_int_to_decimal(price_fixed * (1 + quote_bps / 10000))

# Set order options
expiry_ts = int((datetime.now() + timedelta(seconds=time_in_force)).timestamp())
order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.Limit, client_order_id=69)

# Execute quote!
await client.replace_quote(Asset.SOL, bid_price, quote_size, ask_price, quote_size, order_opts)

INFO:zeta_py.client.Client:Replacing SOL orders: 0.001x Bid @ $17.973859149, 0.001x Ask @ $18.009842851


Signature(
    4JJqnar2gQMZszwJjhK4A19xPLdJ1DX1LQNViueHTCVFky8nRT8qreYRkKtgV5MfPFVBPFjouPrUroUZAW2i3f7T,
)

### Transaction Events

In [3]:
from zeta_py.events import LiquidationEvent, OrderCompleteEvent, TradeEventV3


async def order_complete_callback(event: OrderCompleteEvent):
    print(f"Order complete: {event}")


async def trade_callback(event: TradeEventV3):
    print(f"Trade: {event}")


async def liquidation_callback(event: LiquidationEvent):
    print(f"Liquidation: {event}")


await client.subscribe_transaction(order_complete_callback, trade_callback, liquidation_callback)

Trade: Event(name='TradeEventV3', data=TradeEventV3(margin_account=Pubkey(
    Hu2yXbPESXyamLdvDemeASCFd3rHnpN2xBDUJpnuG5UX,
), index=137, size=1, cost_of_trades=18498, is_bid=True, client_order_id=0, order_id=347924058420979926623339914, asset=Asset.SOL(), user=Pubkey(
    4mYxEXiFwe3ePEmoj7CYCbiW7wGTojUQCmdPqGaoVaUh,
), is_taker=True, sequence_number=174933582, fee=18))


CancelledError: 